In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc, f1_score, matthews_corrcoef
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import statsmodels.stats.multitest as multi
from scipy.stats import mannwhitneyu

In [2]:
df = pd.read_csv("RNA.Result.csv",sep=';', decimal=',')
df

,PatientID,Tumor,A1BG,A2LD1,A2M,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22
0,TCGA-05-4244-01A-01R-1107-07,1,26.0302,135.5022,9844.7858,130.9015,651.0167,803.5605,3.4907,53.0587,...,221.1642,517.6717,39.794000000000004,292.5212,959.2460000000001,1088.0531,2837.9440000000004,871.2802,575.2683,6.6323
1,TCGA-05-4249-01A-01R-1107-07,1,120.1349,89.0629,25712.6617,578.5562,604.3128,999.9976,266.9021,185.4471,...,198.5893,287.5074,62.1376,493.5597,1179.3275,787.5061,2351.25,1138.117,690.2752,179.9738
2,TCGA-05-4250-01A-01R-1107-07,1,50.8597,151.1332,16943.6359,356.4619,588.5766,754.0413,26.1958,81.9033,...,520.8986,1046.8374,60.0182,365.4149,843.9028,475.17199999999997,5437.4534,1170.5214,532.8691,6.3003
3,TCGA-05-4382-01A-01R-1206-07,1,145.9037,112.0685,23326.2727,554.4677,608.3112,1079.9796,3.5896,46.6643,...,429.6299,753.0902,40.4424,346.7517,946.6872,908.1593,6770.1537,1169.2401,663.8297,35.1777
4,TCGA-05-4384-01A-01R-1755-07,1,127.3671,87.5748,48314.5571,152.7184,665.8522,709.8351,34.8198,100.7941,...,258.3018,153.9401,80.0244,563.2254,1320.7086,778.8638,3341.4783,1737.3244,723.2743,378.1307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,TCGA-91-6831-11A-02R-1858-07,0,62.9634,47.3918,79556.0155,338.5128,374.0289,406.2153,196.4484,88.7902,...,134.2952,54.38399999999999,45.505,405.1054,1053.2741,1169.8113,4502.7747,1066.5927,553.8291,84.3507
570,TCGA-91-6835-11A-01R-1858-07,0,76.7722,55.9789,77953.782,561.7934,604.3819,613.7765,125.8868,108.3503,...,177.7634,85.1771,69.5196,367.6396,1246.3419999999999,894.36,4142.3648,1833.8138,545.5095,144.6759
571,TCGA-91-6836-11A-01R-1858-07,0,79.8192,61.3333,55831.1499,296.1186,560.2033,718.0291,117.1973,135.9489,...,117.9162,61.7239,56.2547,346.1228,1398.555,1154.7845,4739.4605,1862.6564,653.1799,85.9447
572,TCGA-91-6847-11A-01R-1949-07,0,67.6923,61.4389,39359.095,267.8733,540.2715,610.8597,470.5882,105.8824,...,150.2262,64.2534,52.4887,360.181,1340.2715,1035.2941,4609.0498,1458.8235,558.371,62.4434


In [3]:
#zapisuje tablice genow aby pozniej dalo sie przypisac nazwy genow do okreslonych zmiennych w train/test
df_genes = pd.DataFrame(columns = ['Name'])
for col in df:
        if col!="PatientID" and col !="Tumor":
            df_genes = df_genes.append({'Name':col},ignore_index = True)

In [4]:
df_genes.index = np.arange(1, len(df_genes) + 1)
df_genes

,Name
1,A1BG
2,A2LD1
3,A2M
4,A4GALT
5,AAAS
...,...
15801,ZYG11B
15802,ZYX
15803,ZZEF1
15804,ZZZ3


In [5]:
X = df.iloc[:, 2:].values
Y = df.iloc[:, 1:2].values
#splaszczenie listy
Y = np.ravel(Y)

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
fprs, tprs, scores = [], [], []

In [7]:
#oblicza pvalue
def ComputePValue(X_test,Y_test):
    PValue={}
    Stat={}
    alpha=0.05
    dfResult=[]
    dfTempX = pd.DataFrame(X_test)
    dfTempY = pd.DataFrame(Y_test)
    dfTempX['Tumor'] = dfTempY
    df1 = dfTempX[dfTempX['Tumor'] == 1]
    df0 = dfTempX[dfTempX['Tumor'] == 0]
    for col in dfTempX:
        if col!="PatientID" and col !="Tumor":
            Stat[col],PValue[col] = mannwhitneyu(df1[col],df0[col])
            if PValue[col]<=alpha:
                dfResult.append({'Name':col,'PValue':PValue[col]})
    dfResult=pd.DataFrame(dfResult)
    return dfResult


In [8]:
#oblicza Pvalue fdr
def PValue_fdr(dfResult):
    alpha = 0.05
    pval=dfResult['PValue']
    pval=list(pval)
    PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_bh')
    Reject_fdr = PValue_fdr[0]
    PValue_fdr = PValue_fdr[1]
    for index,row in dfResult.iterrows():
        dfResult.at[index, 'reject_fdr_bh'] = Reject_fdr.item(index)
        dfResult.at[index, 'PValue_fdr_bh'] = PValue_fdr.item(index)
    dfResult = dfResult.loc[(dfResult['reject_fdr_bh'] == True)&(dfResult['reject_fdr_bh'] == True)]
    return dfResult

In [9]:
#zwraca tabele z PValue oraz nazwami genów
def GetImportantValues(df,n):
    df_fdr = df.sort_values(by ='PValue_fdr_bh' )
    df_fdr = df_fdr.head(n)
    #for index,row in df_fdr.iterrows():
    #    df_fdr['Name'] = df_genes
    return df_fdr

In [10]:
#zwraca tabele z PValue oraz nazwami genów
def GetRandomImportantValues(df,n):
    df_RIV = df.sample(n)
    #for index,row in df_RIV.iterrows():
    #    df_RIV['Name'] = df_genes
    return df_RIV

In [26]:
#ogranicza test/train do okreslonych zmiennych
def FeatureSelection(X_transposed,df):
    dfTempX = pd.DataFrame(X_transposed)
    #dfTempX.index = np.arange(1, len(dfTempX) + 1)
    dfTempX = dfTempX.loc[df.index, :] 
    dfTempX = dfTempX.T
    dfTempX = dfTempX.to_numpy()
    return dfTempX

In [27]:
#umozliwia wybranie zmiennych
def AdjustData(X_test):
    x = pd.DataFrame(X_test)
    x = x.T
    #x.index = np.arange(1, len(x) + 1)
    return x

In [37]:
ACC_scores = []
ROC_AUC_scores =[]
F1_scores=[]
MCC_scores=[]
dfScore = pd.DataFrame(columns = ['n','ACC','ROC_AUC','F1',"MCC"])
cv = StratifiedKFold(n_splits=3, shuffle = False)

for i in range(2):
    if (i == 0):
        n=10
    if (i == 1):
        n=100
    classifier = RandomForestClassifier(n_estimators=500)
    k=0
    print(n)
    for train_index, test_index in cv.split(X,Y):
        k+=1
        X_train, X_test, Y_train, Y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
        #oversampling
        sm = SMOTE()
        X_train, Y_train = sm.fit_sample(X_train, Y_train)

        #wprowadzic tutaj modul do selekcji zmiennych
        
        tempDF = ComputePValue(X_train,Y_train)
        tempDF = PValue_fdr(tempDF)
        RIV = GetRandomImportantValues(tempDF,n)
        tempDF = GetImportantValues(tempDF,n)
        X_train_transposed = AdjustData(X_train)
        X_test_transposed = AdjustData(X_test)
        X_train = FeatureSelection(X_train_transposed,tempDF)
        X_test = FeatureSelection(X_test_transposed,RIV)
        
        #trenowanie lasu losowego
        classifier.fit(X_train,Y_train)
        Y_pred = classifier.predict(X_test)

        ACC_score = classifier.score(X_test,Y_test)
        ACC_scores.append(ACC_score)

        Y_score = classifier.predict_proba(X_test)[:,1]
        
        ROC_AUC_score = roc_auc_score(Y_test,Y_score)
        ROC_AUC_scores.append(ROC_AUC_score)
            
        F1_score = f1_score(Y_test, Y_pred)
        F1_scores.append(F1_score)
        print(confusion_matrix(Y_test, Y_pred))
        MCC_score = matthews_corrcoef(Y_test,Y_pred)
        MCC_scores.append(MCC_score)
        print('split: %i ACC: %.5f, ROC_AUC: %.5f, F1: %.5f, MCC: %.5f' %(k, ACC_score, ROC_AUC_score, F1_score, MCC_score))       
    if(dfScore['n'].any()!=n):
        dfScore = dfScore.append({'n':n,'ACC':np.mean(ACC_scores), 'ROC_AUC':np.mean(ROC_AUC_scores),'F1':np.mean(F1_scores),'MCC':np.mean(MCC_scores)},ignore_index = True)    

10
[[  0  20]
 [  0 172]]
split: 1 ACC: 0.89583, ROC_AUC: 0.32282, F1: 0.94505, MCC: 0.00000


C:\Users\Macie\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[[  0  19]
 [  0 172]]
split: 2 ACC: 0.90052, ROC_AUC: 0.42243, F1: 0.94766, MCC: 0.00000


C:\Users\Macie\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[[  0  20]
 [  0 171]]
split: 3 ACC: 0.89529, ROC_AUC: 0.26798, F1: 0.94475, MCC: 0.00000
100


...

C:\Users\Macie\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[[  0  19]
 [  0 172]]
split: 2 ACC: 0.90052, ROC_AUC: 0.76178, F1: 0.94766, MCC: 0.00000


C:\Users\Macie\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[[  0  20]
 [  0 171]]
split: 3 ACC: 0.89529, ROC_AUC: 0.36038, F1: 0.94475, MCC: 0.00000


C:\Users\Macie\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [29]:
print("ACC:     " + str(np.mean(ACC_scores)))
print("ROC_AUC: " + str(np.mean(ROC_AUC_scores)))
print("F1:      " + str(np.mean(F1_scores)))
print("MCC:     " + str(np.mean(MCC_scores)))

ACC:     0.894601694299011
ROC_AUC: 0.5507879436964505
F1:      0.9443601299829325
MCC:     -0.009975685123665886


In [30]:
dfScore

,n,ACC,ROC_AUC,F1,MCC
0,10.0,0.891988,0.545942,0.942899,-0.019951
1,100.0,0.894602,0.550788,0.944360,-0.009976


In [31]:
tempDF

,Name,PValue,reject_fdr_bh,PValue_fdr_bh
10171,PIGR,1.466439e-113,True,9.096059e-111
3869,DHRS9,6.305596e-114,True,9.096059e-111
6765,ITGB1,6.760499e-114,True,9.096059e-111
4035,DNMBP,4.608279e-114,True,9.096059e-111
10982,PUS3,1.479243e-113,True,9.096059e-111
...,...,...,...,...
10160,PIGC,6.324825e-111,True,9.472744e-109
6227,HM13,6.489942e-111,True,9.619834e-109
7608,LOC219347,6.659361e-111,True,9.770234e-109
5341,FUT6,7.197610e-111,True,1.045326e-108


In [32]:
RIV

,Name,PValue,reject_fdr_bh,PValue_fdr_bh
12243,SFTPB,2.277029e-73,True,1.462221e-72
12138,SEPT8,8.693645e-88,True,9.659755e-87
1745,C1orf109,1.841155e-55,True,7.137267e-55
11257,RBM19,1.566480e-77,True,1.162170e-76
8976,NBEAL2,2.233073e-17,True,3.622603e-17
...,...,...,...,...
423,ALDH8A1,6.938091e-34,True,1.623366e-33
11567,RNF219,5.474331e-84,True,5.229896e-83
695,APOM,3.898605e-03,True,4.160480e-03
10735,PRMT10,4.391922e-28,True,9.040380e-28


In [33]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.411036,0.049569,0.266876,0.220414,0.315960,0.030442,0.175772,0.020495,0.146353,0.251593,...,0.307185,0.178363,0.018641,0.098251,0.191489,0.208470,0.142337,0.020160,0.044619,0.050455
1,0.622165,0.141304,0.175576,0.254421,0.387459,0.087494,0.134620,0.031792,0.196219,0.138789,...,0.484979,0.022135,0.065559,0.053658,0.076119,0.139202,0.426425,0.017441,0.173458,0.042314
2,0.463652,0.149532,0.147769,0.125400,0.231021,0.043472,0.304956,0.019328,0.128745,0.394516,...,0.141129,0.035519,0.086503,0.197814,0.095767,0.111954,0.266394,0.084960,0.032255,0.086296
3,0.387628,0.146974,0.414107,0.111172,0.264487,0.052930,0.266391,0.015143,0.147278,0.191159,...,0.255105,0.123791,0.136949,0.040784,0.034103,0.077252,0.209109,0.093193,0.080491,0.069379
4,0.208209,0.223728,0.166364,0.092263,0.218613,0.181326,0.083512,0.050912,0.295463,0.167257,...,0.325038,0.084765,0.037520,0.129838,0.066577,0.126570,0.304573,0.015785,0.031192,0.023919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,0.270036,0.116469,0.157030,0.047613,0.157331,0.191381,0.642871,0.297276,0.078563,0.261103,...,0.148278,0.162389,0.085914,0.108083,0.035220,0.165055,0.152161,0.229006,0.042354,0.507234
684,0.292889,0.130390,0.224431,0.051448,0.166303,0.130510,0.492175,0.340850,0.070658,0.235671,...,0.149261,0.154265,0.081629,0.119570,0.025155,0.138264,0.125666,0.147986,0.053134,0.341002
685,0.370407,0.087827,0.317770,0.109521,0.060463,0.147324,0.254731,0.183422,0.121185,0.183255,...,0.277411,0.042763,0.105592,0.092988,0.038591,0.070211,0.164996,0.072231,0.083276,0.044900
686,0.203980,0.060228,0.225885,0.112158,0.134960,0.023544,0.230765,0.530689,0.175431,0.166396,...,0.370798,0.072235,0.034168,0.088648,0.070700,0.059100,0.490650,0.162312,0.089078,0.049331


In [34]:
a = pd.DataFrame(Y_pred)
a[0].value_counts()

1    191
Name: 0, dtype: int64

In [35]:
b = pd.DataFrame(Y_test)
b[0].value_counts()

1    171
0     20
Name: 0, dtype: int64